# Problem 1: Quadratic Assignment Problem
**Author:** Sergio López Baños (/LopezBanos)

In [37]:
import dimod           # D-WAVE share API for samplers [1]
import neal
import tabu
import numpy as np     # Python Scientific Package     [2]
import pandas as pd    # Pandas (Data management)      [3]
from utils import path_for_any_os
from dimod import ConstrainedQuadraticModel,BinaryQuadraticModel, Binary, quicksum

## Loading and Cleaning Data
I defined a function in `utils.py` that allow me to read the instances.txt files for any system as long as they are saved in the same way I did, i.e.,  <br>
*instances -> QAP -> example.txt*

In [38]:
NAME='Cornell_example.txt'
if 'dre' in NAME:
    skip = 1
else: 
    skip = 2
# Path to file
PATH = path_for_any_os('QAP', NAME)

# Optimal Value
first_line = pd.read_csv(PATH, delim_whitespace=True, nrows=1, header=None).iloc[0].to_list()
if len(first_line)==2:
    optimal = first_line[1]
else:
    optimal = 10e6
# Problem size
S = int(first_line[0])

# Flow Matrix
F = pd.read_csv(PATH, skiprows=skip, delim_whitespace=True, nrows=S, header=None).to_numpy()

# Distance Matrix
D = pd.read_csv(PATH, skiprows=S+skip+1, delim_whitespace=True, nrows=S, header=None).to_numpy()

# TIMEOUT: Total time of annealing execution
ONE_MIN = 60000
TIMEOUT = ONE_MIN + (S//10)*ONE_MIN # Add an extra minute of execution time every 10 more variables

In [39]:
# Convert to matrix
D = np.asmatrix(D)
F = np.asmatrix(F)

The objective function is given by
\begin{equation}
\min_{\textbf{x}}\sum_{i=1}^{N}\sum_{j=1}^{N}\sum_{p=1}^{N}\sum_{q=1}^{N} f_{ij}d_{pq}x_{ip}x_{jq}
\end{equation}
subject to,
\begin{align}
\sum_{i=1}^{N}x_{ij} = 1, \quad \forall j = 0,..., 1 \\
\sum_{j=1}^{N}x_{ij} = 1, \quad \forall i = 0,..., 1
\end{align}
The meaning of the last two constraints is,
- A given location $i$ can only have one facility $j$.<br>
- A facility cannot be in multiple locations. <br>
To get insight of the problem formulation, one can either formulate a small instance or do some research. At **[4]** I found a compact formulation of the problem that allows me to use numpy speed-up.
\begin{equation}
\tag{Koopmans-Beckmann}
\boxed{\langle  \; F, XDX^{T} \; \rangle = \sum_{i=1}^{N}\sum_{j=1}^{N}\sum_{p=1}^{N}\sum_{q=1}^{N} f_{ij}d_{pq}x_{ip}x_{jq}} 
\end{equation}

##### <font color='red'>Warning: </font>For large instances formulate BQM and go to SimulatedAnnealingSampler

## Build the CQM

In [40]:
# Matrix X of binary variables
X = np.empty([S,S],dtype=object)
for i in range(S):
    for j in range(S):
        X[i,j] = Binary('x_{}_{}'.format(i+1,j+1))
X = np.asmatrix(X)

In [41]:
# Build the CQM
cqm = ConstrainedQuadraticModel()

In [42]:
#B = F.T@X@(D@X.T) # Can we improve it?
B = X@(D@X.T)

In [43]:
# Too slow
#B = np.empty([S,S], dtype=object)
#for row in range(S):
#    for column in range(S):
#        B[row, column] = quicksum(X[row,i]*D[i,j]*X.T[j,column] for i in range(S) for j in range(S))

In [44]:
F_T = F.T
objective = quicksum(F_T[diag,j]*B[j,diag] for diag in range(S) for j in range(S))

In [45]:
# Too slow because we are adding BinaryQuadraticModels objects
#objective = np.trace(F.T@B)
#objective

In [46]:
# Add the objective to the CQM
cqm.set_objective(objective)

In [47]:
# Set the constraints.
for i in range(S):
    cqm.add_constraint(quicksum(X[i,j] for j in range(S)) == 1)
for j in range(S):
    cqm.add_constraint(quicksum(X[i,j] for i in range(S)) == 1)

In [48]:
bqm, invert = dimod.cqm_to_bqm(cqm)

## Build the BQM

In [13]:
bqm = BinaryQuadraticModel('BINARY') # Initialize the BQM

In [14]:
# For loops are not fast but they are faster than numpy matrix multiplication with dtype=objects
for i in range(S):
    for j in range(S):
        for p in range(S):
            for q in range(S):
                # Cannot have diagonal terms
                if (i==j) and (p==q):
                    continue
                    #bqm.add_linear('x_{}_{}'.format(i+1,p+1), F[i,i]*D[p,p]) # Zero term in diagonals
                else:
                    bqm.add_quadratic('x_{}_{}'.format(i+1,p+1),'x_{}_{}'.format(j+1,q+1), F[i,j]*D[p,q])

In [15]:
# First Constraints
for row in range(S):
    constraint = [('x_{}_{}'.format(row+1,j+1), 1) for j in range(S)]
    bqm.add_linear_equality_constraint(
        constraint,
        constant = -1,
        lagrange_multiplier = 4*optimal)
    
# Second Constraints
for column in range(S):
    constraint = [('x_{}_{}'.format(i+1,column+1), 1) for i in range(S)]
    bqm.add_linear_equality_constraint(
        constraint,
        constant = -1,
        lagrange_multiplier = 4*optimal)

According to D-Wave[5] https://support.dwavesys.com/hc/en-us/community/posts/4403954039831-How-can-we-get-execution-time-of-each-annealing-while-using-Neal-Simulated-Annealing-Sampler-:
> The goal is to find the optimal number of sweeps for which the estimated TTS is minimal: the latter can be estimated knowing the time for one read and the probability of reaching the ground state. In other words, in a plot with num_sweeps on the x-axis and TTS on the y-axis, you would expect a convex function (as shown in the research paper linked below).

Maybe this part is out of the scope of the current project for this reason I will propose a number of sweeps acording to the size of the problem. Notice that the bigger the problem the more number of sweeps but that implies longer times of computation.

#### Neal solver

In [49]:
# We could add a custom beta_schedule for high performance applications
results = neal.sampler.SimulatedAnnealingSampler().sample(bqm, num_reads=1000*S, num_sweeps=2*S*S, num_sweeps_per_beta=S)
#exactsolver = dimod.ExactSolver()
#results = exactsolver.sample(bqm)

#### Tabu solver (If need a timeout function)

In [17]:
# Timing with tabu
# Tiempo en llegar a la mejor solucion (en que momento la alcanzas)
sampler = tabu.TabuSampler()
results = sampler.sample(bqm, timeout=TIMEOUT) # Output the best solution found in the given time

In [50]:
results_table = results.to_pandas_dataframe()
index_columns = ['energy', 'num_occurrences']
for i in range(S):
    for j in range(S):
        index_columns.append('x_{}_{}'.format(i+1,j+1))
results_table = results_table[index_columns]
results_table = results_table.sort_values(by=['energy'])
pd.set_option('display.max_rows', 7)
pd.set_option('display.max_columns', 500)

In [51]:
results_table.head(5)

,energy,num_occurrences,x_1_1,x_1_2,x_1_3,x_2_1,x_2_2,x_2_3,x_3_1,x_3_2,x_3_3
0,173.0,1,0,0,1,0,1,0,1,0,0
1209,173.0,1,0,0,1,0,1,0,1,0,0
1213,173.0,1,0,0,1,0,1,0,1,0,0
1214,173.0,1,0,0,1,0,1,0,1,0,0
1217,173.0,1,0,0,1,0,1,0,1,0,0


## Last Comments
There is a trick to reduce the number of variables and computational resources, the upper triangular trick. This trick take advantage of the symmetry of a given problem (when $x_{ij} = x_{ji}$) to add the lower diagonal elements of a matrix to the upper diagonal elements so that we do only have to consider the upper triangular elements.

## Bibliography

**[1]** *dimod Documentation*: https://readthedocs.org/projects/test-projecttemplate-dimod/downloads/pdf/latest/ <br>
**[2]** *Harris, C.R., Millman, K.J., van der Walt, S.J. et al. Array programming with NumPy. Nature 585, 357–362 (2020). DOI: 10.1038/s41586-020-2649-2.* <br>
**[3]** *McKinney, W. (2010). Data Structures for Statistical Computing in Python. In Proceedings of the Python in Science Conferences. https://doi.org/10.25080/majora-92bf1922-00a* <br>
**[4]** QAP Theory: https://optimization.cbe.cornell.edu/index.php?title=Quadratic_assignment_problem <br>